# Food Delivery Dataset Analysis
## Complete Data Integration and Analysis Pipeline

This notebook performs:
1. Loading data from CSV, JSON, and SQL sources
2. Merging datasets using appropriate joins
3. Comprehensive analysis to answer all questions

## Step 0: Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import json
import re
import sqlite3
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

## Step 1: Load CSV Data (orders.csv)

In [ ]:
# Load orders data from CSV
orders_df = pd.read_csv('/mnt/user-data/uploads/orders.csv')

print("Orders DataFrame Info:")
print(f"Shape: {orders_df.shape}")
print(f"\nColumns: {orders_df.columns.tolist()}")
print(f"\nFirst 5 rows:")
print(orders_df.head())
print(f"\nData types:")
print(orders_df.dtypes)

## Step 2: Load JSON Data (users.json)

In [ ]:
# Load users data from JSON
with open('/mnt/user-data/uploads/users.json', 'r') as f:
    users_data = json.load(f)

users_df = pd.DataFrame(users_data)

print("Users DataFrame Info:")
print(f"Shape: {users_df.shape}")
print(f"\nColumns: {users_df.columns.tolist()}")
print(f"\nFirst 5 rows:")
print(users_df.head())
print(f"\nData types:")
print(users_df.dtypes)

## Step 3: Load SQL Data (restaurants.sql)

In [ ]:
# Read SQL file and extract INSERT statements
with open('/mnt/user-data/uploads/restaurants.sql', 'r') as f:
    sql_content = f.read()

# Parse INSERT statements
insert_pattern = r"INSERT INTO restaurants VALUES \((\d+), '([^']+)', '([^']+)', ([\d.]+)\);"
matches = re.findall(insert_pattern, sql_content)

# Create DataFrame from parsed data
restaurants_df = pd.DataFrame(matches, columns=['restaurant_id', 'restaurant_name', 'cuisine', 'rating'])

# Convert data types
restaurants_df['restaurant_id'] = restaurants_df['restaurant_id'].astype(int)
restaurants_df['rating'] = restaurants_df['rating'].astype(float)

print("Restaurants DataFrame Info:")
print(f"Shape: {restaurants_df.shape}")
print(f"\nColumns: {restaurants_df.columns.tolist()}")
print(f"\nFirst 5 rows:")
print(restaurants_df.head())
print(f"\nData types:")
print(restaurants_df.dtypes)

## Step 4: Merge the Datasets

Performing LEFT JOINS:
1. orders + users (on user_id)
2. result + restaurants (on restaurant_id)

In [ ]:
# First merge: orders with users (LEFT JOIN)
merged_df = pd.merge(
    orders_df,
    users_df,
    on='user_id',
    how='left'
)

print("After merging with users:")
print(f"Shape: {merged_df.shape}")
print(f"\nFirst 3 rows:")
print(merged_df.head(3))

In [ ]:
# Second merge: result with restaurants (LEFT JOIN)
final_df = pd.merge(
    merged_df,
    restaurants_df,
    on='restaurant_id',
    how='left',
    suffixes=('_order', '_rest')
)

print("Final Merged DataFrame Info:")
print(f"Shape: {final_df.shape}")
print(f"\nColumns: {final_df.columns.tolist()}")
print(f"\nFirst 3 rows:")
print(final_df.head(3))
print(f"\nData types:")
print(final_df.dtypes)

## Step 5: Create Final Dataset

In [ ]:
# Parse order_date to datetime
final_df['order_date'] = pd.to_datetime(final_df['order_date'], format='%d-%m-%Y')

# Add additional date features for analysis
final_df['year'] = final_df['order_date'].dt.year
final_df['month'] = final_df['order_date'].dt.month
final_df['quarter'] = final_df['order_date'].dt.quarter

print("Final Dataset Summary:")
print(f"Total rows: {len(final_df)}")
print(f"Total columns: {len(final_df.columns)}")
print(f"\nColumns in final dataset:")
for col in final_df.columns:
    print(f"  - {col}")

print("\nSample of final dataset:")
print(final_df.head())

In [ ]:
# Save the final merged dataset
output_path = '/mnt/user-data/outputs/final_food_delivery_dataset.csv'
final_df.to_csv(output_path, index=False)
print(f"Final dataset saved to: {output_path}")
print(f"Total records: {len(final_df)}")

## Data Quality Checks

In [ ]:
# Check for null values
print("Null values in final dataset:")
print(final_df.isnull().sum())

print("\n" + "="*50)
print("Basic Statistics:")
print("="*50)
print(f"Total Orders: {len(final_df)}")
print(f"Unique Users: {final_df['user_id'].nunique()}")
print(f"Unique Restaurants: {final_df['restaurant_id'].nunique()}")
print(f"Date Range: {final_df['order_date'].min()} to {final_df['order_date'].max()}")
print(f"Total Revenue: ₹{final_df['total_amount'].sum():,.2f}")

---
# FILL IN THE BLANKS - ANSWERS

In [ ]:
print("="*60)
print("FILL IN THE BLANKS - ANSWERS")
print("="*60)

# Q1: The column used to join orders.csv and users.json
answer_1 = "user_id"
print(f"1. The column used to join orders.csv and users.json is: {answer_1}")

# Q2: The dataset containing cuisine and rating information is stored in __ format
answer_2 = "SQL"
print(f"2. The dataset containing cuisine and rating information is stored in: {answer_2} format")

# Q3: The total number of rows in the final merged dataset
answer_3 = len(final_df)
print(f"3. The total number of rows in the final merged dataset is: {answer_3}")

# Q4: If a user has no matching record in users.json, the merged values will be
answer_4 = "NaN (or null)"
print(f"4. If a user has no matching record in users.json, the merged values will be: {answer_4}")

# Q5: The Pandas function used to combine datasets based on a key
answer_5 = "merge"
print(f"5. The Pandas function used to combine datasets based on a key is: {answer_5}")

# Q6: The column membership originates from the __ file
answer_6 = "users.json"
print(f"6. The column membership in the final dataset originates from the: {answer_6} file")

# Q7: The join key used to combine orders data with restaurant details
answer_7 = "restaurant_id"
print(f"7. The join key used to combine orders data with restaurant details is: {answer_7}")

# Q8: The column that helps identify the type of food served by a restaurant
answer_8 = "cuisine"
print(f"8. The column that helps identify the type of food served by a restaurant is: {answer_8}")

# Q9: If a user places multiple orders, their personal details appear __ times
answer_9 = "multiple (equal to the number of orders they placed)"
print(f"9. If a user places multiple orders, their personal details appear: {answer_9}")

---
# SHORT ANSWER QUESTIONS

In [ ]:
print("="*60)
print("SHORT ANSWER QUESTIONS")
print("="*60)

# Q1: How many total orders were placed by users with Gold membership?
gold_orders = final_df[final_df['membership'] == 'Gold'].shape[0]
print(f"\n1. Total orders by Gold members: {gold_orders}")

# Q2: Total revenue from orders placed in Hyderabad city (rounded to nearest integer)
hyderabad_revenue = final_df[final_df['city'] == 'Hyderabad']['total_amount'].sum()
print(f"2. Total revenue from Hyderabad: ₹{round(hyderabad_revenue)}")

# Q3: How many distinct users placed at least one order?
distinct_users = final_df['user_id'].nunique()
print(f"3. Distinct users who placed at least one order: {distinct_users}")

# Q4: Average order value for Gold members (rounded to 2 decimals)
gold_avg_order = final_df[final_df['membership'] == 'Gold']['total_amount'].mean()
print(f"4. Average order value for Gold members: ₹{round(gold_avg_order, 2)}")

# Q5: How many orders were placed for restaurants with rating >= 4.5?
high_rating_orders = final_df[final_df['rating'] >= 4.5].shape[0]
print(f"5. Orders placed for restaurants with rating >= 4.5: {high_rating_orders}")

# Q6: How many orders were placed in the top revenue city among Gold members only?
gold_members_df = final_df[final_df['membership'] == 'Gold']
city_revenue = gold_members_df.groupby('city')['total_amount'].sum().sort_values(ascending=False)
top_revenue_city = city_revenue.index[0]
orders_in_top_city = gold_members_df[gold_members_df['city'] == top_revenue_city].shape[0]
print(f"6. Top revenue city among Gold members: {top_revenue_city}")
print(f"   Orders placed in {top_revenue_city} by Gold members: {orders_in_top_city}")

---
# MULTIPLE CHOICE QUESTIONS - ANALYSIS

### Q1: Which city has the highest total revenue from Gold members?

In [ ]:
# Filter for Gold members
gold_df = final_df[final_df['membership'] == 'Gold']

# Calculate revenue by city for Gold members
city_revenue_gold = gold_df.groupby('city')['total_amount'].sum().sort_values(ascending=False)

print("Revenue by City (Gold Members):")
print(city_revenue_gold)
print(f"\n✓ ANSWER: {city_revenue_gold.index[0]}")

### Q2: Which cuisine has the highest average order value?

In [ ]:
# Calculate average order value by cuisine
cuisine_avg = final_df.groupby('cuisine')['total_amount'].mean().sort_values(ascending=False)

print("Average Order Value by Cuisine:")
print(cuisine_avg)
print(f"\n✓ ANSWER: {cuisine_avg.index[0]}")

### Q3: How many distinct users placed orders worth more than ₹1000 in total?

In [ ]:
# Calculate total amount per user
user_total_spending = final_df.groupby('user_id')['total_amount'].sum()

# Count users with total spending > 1000
users_above_1000 = (user_total_spending > 1000).sum()

print(f"Users who spent more than ₹1000 in total: {users_above_1000}")

# Determine range
if users_above_1000 < 500:
    answer = "< 500"
elif users_above_1000 <= 1000:
    answer = "500 – 1000"
elif users_above_1000 <= 2000:
    answer = "1000 – 2000"
else:
    answer = "> 2000"

print(f"\n✓ ANSWER: {answer}")

### Q4: Which restaurant rating range generated the highest total revenue?

In [ ]:
# Create rating bins
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ['3.0 – 3.5', '3.6 – 4.0', '4.1 – 4.5', '4.6 – 5.0']

final_df['rating_range'] = pd.cut(final_df['rating'], bins=bins, labels=labels, include_lowest=True)

# Calculate revenue by rating range
rating_revenue = final_df.groupby('rating_range', observed=True)['total_amount'].sum().sort_values(ascending=False)

print("Revenue by Rating Range:")
print(rating_revenue)
print(f"\n✓ ANSWER: {rating_revenue.index[0]}")

### Q5: Among Gold members, which city has the highest average order value?

In [ ]:
# Filter for Gold members and calculate average order value by city
gold_city_avg = gold_df.groupby('city')['total_amount'].mean().sort_values(ascending=False)

print("Average Order Value by City (Gold Members):")
print(gold_city_avg)
print(f"\n✓ ANSWER: {gold_city_avg.index[0]}")

### Q6: Which cuisine has the lowest number of distinct restaurants but still contributes significant revenue?

In [ ]:
# Calculate distinct restaurants and revenue by cuisine
cuisine_stats = final_df.groupby('cuisine').agg({
    'restaurant_id': 'nunique',
    'total_amount': 'sum'
}).rename(columns={'restaurant_id': 'distinct_restaurants', 'total_amount': 'total_revenue'})

cuisine_stats = cuisine_stats.sort_values('distinct_restaurants')

print("Cuisine Analysis (sorted by number of distinct restaurants):")
print(cuisine_stats)
print(f"\n✓ ANSWER: {cuisine_stats.index[0]} (lowest restaurants, but still significant revenue)")

### Q7: What percentage of total orders were placed by Gold members?

In [ ]:
# Calculate percentage of Gold member orders
total_orders = len(final_df)
gold_orders = len(final_df[final_df['membership'] == 'Gold'])
gold_percentage = (gold_orders / total_orders) * 100

print(f"Total Orders: {total_orders}")
print(f"Gold Member Orders: {gold_orders}")
print(f"Percentage: {gold_percentage:.2f}%")
print(f"Rounded: {round(gold_percentage)}%")
print(f"\n✓ ANSWER: {round(gold_percentage)}%")

### Q8: Which restaurant has the highest average order value but less than 20 total orders?

In [ ]:
# Group by restaurant and calculate stats
restaurant_stats = final_df.groupby('restaurant_name_rest').agg({
    'total_amount': ['mean', 'count']
}).round(2)

restaurant_stats.columns = ['avg_order_value', 'total_orders']

# Filter for restaurants with less than 20 orders
low_order_restaurants = restaurant_stats[restaurant_stats['total_orders'] < 20]
low_order_restaurants = low_order_restaurants.sort_values('avg_order_value', ascending=False)

print("Top 10 Restaurants with < 20 orders (by average order value):")
print(low_order_restaurants.head(10))
print(f"\n✓ ANSWER: {low_order_restaurants.index[0]}")

### Q9: Which membership + cuisine combination contributes the highest revenue?

In [ ]:
# Calculate revenue by membership and cuisine combination
combo_revenue = final_df.groupby(['membership', 'cuisine'])['total_amount'].sum().sort_values(ascending=False)

print("Revenue by Membership + Cuisine Combination:")
print(combo_revenue)

top_combo = combo_revenue.index[0]
print(f"\n✓ ANSWER: {top_combo[0]} + {top_combo[1]} cuisine")

### Q10: During which quarter of the year is the total revenue highest?

In [ ]:
# Calculate revenue by quarter
quarter_revenue = final_df.groupby('quarter')['total_amount'].sum().sort_values(ascending=False)

quarter_names = {
    1: 'Q1 (Jan–Mar)',
    2: 'Q2 (Apr–Jun)',
    3: 'Q3 (Jul–Sep)',
    4: 'Q4 (Oct–Dec)'
}

print("Revenue by Quarter:")
for quarter, revenue in quarter_revenue.items():
    print(f"{quarter_names[quarter]}: ₹{revenue:,.2f}")

top_quarter = quarter_revenue.index[0]
print(f"\n✓ ANSWER: {quarter_names[top_quarter]}")

---
# SUMMARY OF ALL ANSWERS

In [ ]:
print("="*80)
print("COMPLETE ANSWER KEY")
print("="*80)

print("\n📝 FILL IN THE BLANKS:")
print("-" * 80)
print("1. user_id")
print("2. SQL")
print(f"3. {len(final_df)}")
print("4. NaN (or null)")
print("5. merge")
print("6. users.json")
print("7. restaurant_id")
print("8. cuisine")
print("9. multiple (equal to the number of orders they placed)")

print("\n📊 SHORT ANSWER QUESTIONS:")
print("-" * 80)
gold_orders = final_df[final_df['membership'] == 'Gold'].shape[0]
hyderabad_revenue = final_df[final_df['city'] == 'Hyderabad']['total_amount'].sum()
distinct_users = final_df['user_id'].nunique()
gold_avg_order = final_df[final_df['membership'] == 'Gold']['total_amount'].mean()
high_rating_orders = final_df[final_df['rating'] >= 4.5].shape[0]

gold_members_df = final_df[final_df['membership'] == 'Gold']
city_revenue_summary = gold_members_df.groupby('city')['total_amount'].sum().sort_values(ascending=False)
top_revenue_city = city_revenue_summary.index[0]
orders_in_top_city = gold_members_df[gold_members_df['city'] == top_revenue_city].shape[0]

print(f"1. {gold_orders}")
print(f"2. ₹{round(hyderabad_revenue)}")
print(f"3. {distinct_users}")
print(f"4. ₹{round(gold_avg_order, 2)}")
print(f"5. {high_rating_orders}")
print(f"6. {orders_in_top_city}")

print("\n✅ MULTIPLE CHOICE QUESTIONS:")
print("-" * 80)

# Recalculate all MCQ answers
gold_df = final_df[final_df['membership'] == 'Gold']
city_revenue_gold = gold_df.groupby('city')['total_amount'].sum().sort_values(ascending=False)
cuisine_avg = final_df.groupby('cuisine')['total_amount'].mean().sort_values(ascending=False)
user_total_spending = final_df.groupby('user_id')['total_amount'].sum()
users_above_1000 = (user_total_spending > 1000).sum()

if users_above_1000 < 500:
    answer_q3 = "< 500"
elif users_above_1000 <= 1000:
    answer_q3 = "500 – 1000"
elif users_above_1000 <= 2000:
    answer_q3 = "1000 – 2000"
else:
    answer_q3 = "> 2000"

bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ['3.0 – 3.5', '3.6 – 4.0', '4.1 – 4.5', '4.6 – 5.0']
final_df['rating_range'] = pd.cut(final_df['rating'], bins=bins, labels=labels, include_lowest=True)
rating_revenue = final_df.groupby('rating_range', observed=True)['total_amount'].sum().sort_values(ascending=False)

gold_city_avg = gold_df.groupby('city')['total_amount'].mean().sort_values(ascending=False)

cuisine_stats = final_df.groupby('cuisine').agg({
    'restaurant_id': 'nunique',
    'total_amount': 'sum'
}).rename(columns={'restaurant_id': 'distinct_restaurants', 'total_amount': 'total_revenue'})
cuisine_stats = cuisine_stats.sort_values('distinct_restaurants')

total_orders = len(final_df)
gold_orders_pct = len(final_df[final_df['membership'] == 'Gold'])
gold_percentage = (gold_orders_pct / total_orders) * 100

restaurant_stats = final_df.groupby('restaurant_name_rest').agg({
    'total_amount': ['mean', 'count']
}).round(2)
restaurant_stats.columns = ['avg_order_value', 'total_orders']
low_order_restaurants = restaurant_stats[restaurant_stats['total_orders'] < 20]
low_order_restaurants = low_order_restaurants.sort_values('avg_order_value', ascending=False)

combo_revenue = final_df.groupby(['membership', 'cuisine'])['total_amount'].sum().sort_values(ascending=False)
top_combo = combo_revenue.index[0]

quarter_revenue = final_df.groupby('quarter')['total_amount'].sum().sort_values(ascending=False)
quarter_names = {
    1: 'Q1 (Jan–Mar)',
    2: 'Q2 (Apr–Jun)',
    3: 'Q3 (Jul–Sep)',
    4: 'Q4 (Oct–Dec)'
}
top_quarter = quarter_revenue.index[0]

print(f"1. {city_revenue_gold.index[0]}")
print(f"2. {cuisine_avg.index[0]}")
print(f"3. {answer_q3}")
print(f"4. {rating_revenue.index[0]}")
print(f"5. {gold_city_avg.index[0]}")
print(f"6. {cuisine_stats.index[0]}")
print(f"7. {round(gold_percentage)}%")
print(f"8. {low_order_restaurants.index[0]}")
print(f"9. {top_combo[0]} + {top_combo[1]} cuisine")
print(f"10. {quarter_names[top_quarter]}")

print("\n" + "="*80)
print("Analysis Complete!")
print("="*80)

---
## Additional Insights and Visualizations

In [ ]:
# Summary statistics
print("\n📈 KEY INSIGHTS:")
print("="*60)
print(f"Total Revenue: ₹{final_df['total_amount'].sum():,.2f}")
print(f"Average Order Value: ₹{final_df['total_amount'].mean():.2f}")
print(f"Median Order Value: ₹{final_df['total_amount'].median():.2f}")
print(f"\nMembership Distribution:")
print(final_df['membership'].value_counts())
print(f"\nCity Distribution:")
print(final_df['city'].value_counts())
print(f"\nCuisine Distribution:")
print(final_df['cuisine'].value_counts())
print(f"\nRating Statistics:")
print(final_df['rating'].describe())